In [43]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np

In [44]:
project_name = 'Split_Sys_VQA'
exp_name = 'Naive'
# exp_name = 'ER'
# exp_name = 'LwF'
# exp_name = 'GEM'

In [45]:
results = np.load(os.path.join('../../avalanche-experiments', project_name, exp_name, f'results-Novel-{exp_name}.npy'), allow_pickle=True)
print(results)

[{'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.6, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 1.5807034373283386, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.6, 'Loss_Stream/eval_phase/test_stream/Task000': 1.5807034373283386, 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 1.0, 'Loss_Epoch/train_phase/train_stream/Task000': 0.005347466601190371}
 {'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.6, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 1.5807034373283386, 'Top1_Acc_Stream/eval_phase/test_stream/Task000': 0.81, 'Loss_Stream/eval_phase/test_stream/Task000': 0.7675045132637024, 'Top1_Acc_Epoch/train_phase/train_stream/Task000': 1.0, 'Loss_Epoch/train_phase/train_stream/Task000': 0.0016083522454007633, 'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001': 0.81, 'Loss_Exp/eval_phase/test_stream/Task000/Exp001': 0.7675045132637024}
 {'Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000': 0.6, 'Loss_Exp/eval_phase/test_stream/Task000/Exp000': 1.5

In [46]:
accs = [result['Top1_Acc_Stream/eval_phase/test_stream/Task000'] for result in results]
print(accs)

[0.6, 0.81, 0.65, 0.77, 0.78, 0.81, 0.62, 0.77, 0.81, 0.66]


In [47]:
print(f'{np.mean(accs)*100:.2f}% ({np.std(accs)*100:.2f}%)')

72.80% (8.07%)
